In [ ]:
from library.matrix import truncate_p as truncate

# Q1: Use Midpoint, Trapezoidal and Simpson numerical integration scheme to evaluate the integral:

$$\int_1^4 \sqrt{1+\frac{1}{x}}\;dx$$

In [20]:
from library.integration import simpson_rule, trapezoidal_rule, midpoint_rule, monte_carlo_integration
def func(x):
    return (1+1/x)**0.5

In [28]:
print("Midpoint Scheme (n=10):", midpoint_rule(func, 1, 4, n=10))
print("Midpoint Scheme (n=20):", midpoint_rule(func, 1, 4, n=20))
print("Midpoint Scheme (n=30):", midpoint_rule(func, 1, 4, n=30))

Midpoint Scheme (n=10): 3.618978893939812
Midpoint Scheme (n=20): 3.6198800323016482
Midpoint Scheme (n=30): 3.62004881243588


In [27]:
print("Trapezoidal Scheme (n=10):", trapezoidal_rule(func, 1, 4, n=10))
print("Trapezoidal Scheme (n=20):", trapezoidal_rule(func, 1, 4, n=20))
print("Trapezoidal Scheme (n=30):", trapezoidal_rule(func, 1, 4, n=30))

Trapezoidal Scheme (n=10): 3.622608380359955
Trapezoidal Scheme (n=20): 3.620793637149885
Trapezoidal Scheme (n=30): 3.6204553882819273


In [25]:
print("Simpson Scheme (n=10):", simpson_rule(func, 1, 4, n=10))
print("Simpson Scheme (n=20):", simpson_rule(func, 1, 4, n=20))
print("Simpson Scheme (n=30):", simpson_rule(func, 1, 4, n=30))

Simpson Scheme (n=10): 3.620188722746527
Simpson Scheme (n=20): 3.62018456725106
Simpson Scheme (n=30): 3.620184337717895


## Comparison of the three methods
|Steps|Midpoint|Trapezoidal|Simpson|
|:---:|:---:|:---:|:---:|
|10|3.618978|3.622608|3.620188|
|20|3.619880|3.620793|3.620184|
|30|3.620048|3.620455|3.620184|

# Q2: Monte Carlo Integration for the following integral:

$$\int_{-1}^1\sin^2(x)\;dx$$

In [29]:
from math import sin
def func2(x):
    return sin(x)**2

In [37]:
monte_carlo_integration(func2, -1, 1, n=5e6, seed=0.49, verbose = True)

100%|██████████| 5000000/5000000 [00:07<00:00, 697426.13it/s]


0.545402579349325

# Q3: A 2 meter long beam has a linear mass density $\lambda(x) = x^2$,  where x is measured from one its ends. Find the center of mass of the beam numerically.

I have solved this problem in two methods

In [1]:
from library.matrix import truncate_p as truncate

## A. Naive approach:
Here I (as a human) have to calculate a lot of things and simplify it to a single expression. <br>
This approach guarantees that the result is correct and deterministically reaches the answer. <br>
But here, the computer does not do everything for us, we have to do a lot of calculations ourselves.

The formula for the center of mass is:

$$x_{COM} = \frac{\sum m_i x_i}{\sum m_i} = \frac{\int_0^M x dm}{M}\;\;\;\;\;[M = total\;mass]$$

now, mass density is given to be $\lambda(x) = x^2$, so the center of mass will be at:

$$x_{COM} = \frac{\int \lambda x dx}{\int xdx}$$

Putting the value of $\lambda(x)=x^2$ in the above equation, we get:

$$x_{COM} = \frac{\int_0^2 x^3 dx}{\int_0^2 x^2dx}$$



In [2]:
lower_limit = 0
upper_limit = 2
steps = 3
a = simpson_rule(lambda x: x**3, lower_limit, upper_limit, steps)
b = simpson_rule(lambda x: x**2, lower_limit, upper_limit, steps)
x_COM = a/b
print(f"Center of mass is at x = {x_COM}m")
# value was supposed to come out to be x = 1.5m.
# The small error can be avoided if we took more steps

Center of mass is at x = 1.4999999999999998m


## B. More intuitive approach

Lazy me wanted my computer to do all the work

### Aim:
&emsp;To find the center of mass of the beam with mass density $\lambda(x) = x^2$.

### Theory:
&emsp;Let us use the fact that, if a body is pivoted at its center of mass, it will not rotate (net torque on it will be zero). So, we will try to find the net torque on the body if pivoted at a random position $a$. Then we will minimise the net torque (minimum value will be close to 0) by adjusting the $a$ to find the center of mass. This idea is inspired from the linear regression algorithm.

### Method:
 - start at a random location on the beam (we have started with $a = 0$)
 - find torque on the beam if pivoted at that location (point $a$)
 - change the value of $a$ depending on the torque (if torque is positive, move a to the right, if negative, move a to the left. Also the distance by which we move $a$ is proportional (proportionality constant is $\alpha$) to the torque)
 - with every step, we will reach a point where the torque will be lesser and lesser. Thus we will keep on taking smaller and smaller steps until we reach a point where the torque is almost zero. This point will be the center of mass of the beam. (**Note:** If we take a large value of $\alpha$, the algorithm may not converge to the correct answer, rather it will diverge out. If we take a small value of $\alpha$, the algorithm will take a lot of time to converge to the correct answer. So, we have to choose a value of $\alpha$ such that the algorithm converges to the correct answer in a reasonable amount of time.)

In [38]:
def get_torque(a):
	f_wrt_a = lambda x: (x-a)*x**2
	torque_left = simpson_rule(f_wrt_a,  0,  a,  n=4)
	torque_right = simpson_rule(f_wrt_a,  a,  2,  n=4)
    # simpson rule acts so good here because the f_wrt_a is a tertiary function
    # and simpson rule is based upon quadratic model... 2 and 3 are close numbers
	return torque_left + torque_right

# Testing the function:
get_torque(2.5)

-2.666666666666667

In [13]:
tollerance = 1e-6
func = lambda x: x**2
alpha = 0.4  # carefully chosen to reduce the number of steps

net_torque = 1  # initiate at any value more than tollerance
a = 0  # initiate at any point on the rod (0, 2)... although it doesn't matter
steps = 0  # for counting the steps
found = True  

net_torques = []
positions = []
while abs(net_torque)>tollerance:
    net_torque = get_torque(a)
    net_torques.append(abs(net_torque))  # just keeping the records
    positions.append(a)  # just keeping the records
    print(f"Step {steps+1}:\ta = {truncate(a, 6, str)},", end = " ")
    a += net_torque*alpha  # Updating a
    print(f"net_torque = {truncate(net_torque, 6, str)}")
    if len(net_torques)-1 and net_torques[-1] > net_torques[-2]:
        print("The net_torque is increasing, the algorithm is diverging, decreasing the value of alpha might help!")
        found = False
        break
    steps += 1

if found: print(f"Center of mass is at x = {truncate(a, 6, str)}m (took {steps} steps)")

Step 1:	a = 0, net_torque = 4.000000
Step 2:	a = 1.600000, net_torque = -0.266666
Step 3:	a = 1.493333, net_torque = 0.017777
Step 4:	a = 1.500444, net_torque = -0.001185
Step 5:	a = 1.499970, net_torque = 0.000079
Step 6:	a = 1.500001, net_torque = -0.000005
Step 7:	a = 1.499999, net_torque = 0.000000
Center of mass is at x = 1.500000m (took 7 steps)


### Demonstration:

&emsp; Dotted lines are only for directing your eyes to the point of interest. The magenta dotted line is the variation of absolute value of torque with $a$. The cyan dotted line denotes the change in $a$ with each iteration. **(points on the of the cyan line don't  mean anything)**

&emsp; Although I am not backing my claim with mathematically rigorous proof, but from the graph we can see that the variation of torque with $a$ is a linear curve with slope $-\frac{8}{3}$ and the curve intersects the x-axis at $a = 1.5$. So, the center of mass is at $a = 1.5$.

#### Here are some graphs for different values of $\alpha$:
(To see the code to generate this graph, visit [here](https://gist.github.com/PeithonKing/dcf64c551cf265cf7ed20901beffe150))

| alpha = 0.65 | alpha = 0.1 | alpha = 0.8 |
|:---:|:---:|:---:|
|![alpha = 0.65](https://github.com/PeithonKing/Some-Static-files/blob/main/ass5_1.png)|![alpha = 0.1](https://github.com/PeithonKing/Some-Static-files/blob/main/ass5_2.png)|![alpha = 0.8](https://github.com/PeithonKing/Some-Static-files/blob/main/ass5_3.png)|